# Examination of effecient NHL roster design

In [62]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pylab import hist, show
import scipy

## Import dataframes

In [63]:
dm = pd.read_csv('t_play_by_play_o.csv')
dm = dm.sort_values(['Season', 'GameNumber', 'Period', 'EventNumber'], ascending=[True, True, True, True])
dm = dm [['Season', 'GameNumber', 'EventNumber', 'Period', 'AdvantageType', 'EventTimeFromZero', 'EventTimeFromTwenty', 'EventType', 'EventDetail', 'VPlayer1', 'VPlayer1Position', 'VPlayer2', 'VPlayer2Position', 'VPlayer3', 'VPlayer3Position', 'VPlayer4', 'VPlayer4Position', 'VPlayer5', 'VPlayer5Position', 'VPlayer6', 'VPlayer6Position', 'HPlayer1', 'HPlayer1Position', 'HPlayer2', 'HPlayer2Position', 'HPlayer3', 'HPlayer3Position', 'HPlayer4', 'HPlayer4Position', 'HPlayer5', 'HPlayer5Position', 'HPlayer6', 'HPlayer6Position']]

In [41]:
dg = pd.read_csv('t_play_by_play_giveaway_detail_o.csv')
dg = dg.rename(columns={'GivePlayerNumber': 'PlayerNumber', 'GivePlayerLName': 'PlayerName', 'GiveTeamCode': 'TeamCode'})

In [42]:
dt = pd.read_csv('t_play_by_play_takeaway_detail_o.csv')
dt = dt.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dt = dt.rename(columns={'TakePlayerNumber': 'PlayerNumber', 'TakePlayerLName': 'PlayerName', 'TakeTeamCode': 'TeamCode'})

In [43]:
dl = pd.read_csv('t_play_by_play_goal_detail_o.csv')
dl = dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dl = dl.rename(columns={'PlayerLName': 'PlayerName'})

In [44]:
ds = pd.read_csv('t_play_by_play_shot_detail_o.csv')
ds = ds.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
ds = ds.rename(columns={'PlayerLName': 'PlayerName'})

In [45]:
dn = pd.read_csv('t_play_by_play_miss_detail_o.csv')
dn = dn.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dn = dn.rename(columns={'PlayerLName': 'PlayerName'})

In [46]:
db = pd.read_csv('t_play_by_play_block_detail_o.csv')
db = db.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
db = db.rename(columns={'ShotPlayerLName': 'ShotPlayerName', 'BlockPlayerLName': 'PlayerName', 'BlockPlayerNumber': 'PlayerNumber', 'BlockTeamCode': 'TeamCode'})

In [47]:
dh = pd.read_csv('t_play_by_play_hit_detail_o.csv')
dh = dh.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dh = dh.rename(columns={'HitterPlayerLName': 'HitterPlayerName', 'HitteePlayerLName': 'HitteePlayerName'})

In [48]:
df = pd.read_csv('t_play_by_play_faceoff_detail_o.csv')
df = df.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
df = df.rename(columns={'VPlayerLName': 'VPlayerName', 'HPlayerLName': 'HPlayerName'})

In [49]:
dp = pd.read_csv('t_play_by_play_penalty_detail_o.csv')
dp = dp.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dp = dp.rename(columns={'PenaltyPlayerLName': 'PenaltyPlayerName', 'DrawnByPlayerLName': 'DrawnByPlayerName'})

In [50]:
dd = pd.read_csv('t_game_detail_o.csv')

# merge event details into play by play

In [52]:
len(dm)

399135

In [53]:
dz = pd.merge(dm,dg, on=['Season', 'GameNumber', 'EventNumber'], how='left')
dz = pd.merge(dm,dt, on=['Season', 'GameNumber', 'EventNumber'], how='left')
dz = pd.merge(dm,ds, on=['Season', 'GameNumber', 'EventNumber'], how='left')
dz = pd.merge(dm,dn, on=['Season', 'GameNumber', 'EventNumber'], how='left')
dz = pd.merge(dm,db, on=['Season', 'GameNumber', 'EventNumber'], how='left')
dz = pd.merge(dm,df, on=['Season', 'GameNumber', 'EventNumber'], how='left')
dz = pd.merge(dm,dp, on=['Season', 'GameNumber', 'EventNumber'], how='left')
dz = pd.merge(dm,dl, on=['Season', 'GameNumber', 'EventNumber'], how='left')
del dl, dp, df, db, dn, ds, dt, dg

In [54]:
len(dm)

399135

In [55]:
dm.columns

Index(['Season', 'GameNumber', 'EventNumber', 'Period', 'AdvantageType',
       'EventTimeFromZero', 'EventTimeFromTwenty', 'EventType', 'EventDetail',
       'VPlayer1', 'VPlayer1Position', 'VPlayer2', 'VPlayer2Position',
       'VPlayer3', 'VPlayer3Position', 'VPlayer4', 'VPlayer4Position',
       'VPlayer5', 'VPlayer5Position', 'VPlayer6', 'VPlayer6Position',
       'HPlayer1', 'HPlayer1Position', 'HPlayer2', 'HPlayer2Position',
       'HPlayer3', 'HPlayer3Position', 'HPlayer4', 'HPlayer4Position',
       'HPlayer5', 'HPlayer5Position', 'HPlayer6', 'HPlayer6Position'],
      dtype='object')

# remove irrelevant observations

In [56]:
dm = dm[dm['GameNumber'] <= 20002]
dm = dm[dm['EventType']!='STOP']

# man-advantage scenarios

In [57]:
value_list = ['PP', 'SH']
dm[dm['AdvantageType'].isin(value_list)]
dm = dm[dm['AdvantageType'] != 'PP']
dm = dm[dm['AdvantageType'] != 'SH']
dm['AdvantageType'] = dm['AdvantageType'].fillna('EV')

# generate time between events

In [58]:
# set_index (unit of observation: season, gn, en)
dm['TimeBetweenEvent'] = dm['EventTimeFromZero'] - dm['EventTimeFromZero'].shift(1)
dm['TimeBetweenEvent'] = dm['TimeBetweenEvent'].abs()

In [68]:
dm.to_csv('pbpmerge.csv', index='False', sep=',')